In [43]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [18]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_la.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

,Unnamed: 0,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,384714,384714,1115657202,4/15/20,679,PPP,"UP PROFESSIONAL SOLUTIONS, LLC",2400 Veterans Memorial Blvd. Suite 400,KENNER,LA,...,999 Other,5613 Employment Services,5613.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,2.205102e+14
1,384715,384715,1623197104,4/10/20,679,PPP,"DYNAMIC INDUSTRIES, INC.",400 Poydras Street Ste 1800 0.0,New Orleans,LA,...,333 Machinery,"3331 Agriculture, Construction, and Mining Mac...",3331.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,2.207101e+14
2,384716,384716,4241517309,4/29/20,679,PPP,"BLESSEY MARINE SERVICE, INC.",1515 River Oaks Road East,harahan,LA,...,999 Other,4832 Inland Water Transportation,4832.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,2.205102e+14
3,384717,384717,5102977301,4/30/20,679,PPP,"JOHN H. CARTER CO., INC.",17630 Perkins Rd,BATON ROUGE,LA,...,332 Fabricated Metal Products,3329 Other Fabricated Metal Product Manufacturing,3329.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,100 to 249,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,2.203300e+14
4,384718,384718,5378948703,4/2/21,679,PPP,KIPP NEW ORLEANS INC,1307 Oretha Castle Haley Blvd Ste 302,New Orleans,LA,...,999 Other,6111 Elementary and Secondary Schools,6111.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13663,398377,398377,9651717708,5/1/20,679,PPP,"QUALITY EXTERIORS, INC.",1000 Chinaberry Dr,Bossier City,LA,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1000 Chinaberry Dr, Bossier City, LA",32.584312,-93.731130,2.201501e+14
13664,398378,398378,9659377206,4/28/20,679,PPP,E-1 ELECTRIC LLC,1440 N. Dorgenois St,NEW ORLEANS,LA,...,999 Other,2382 Building Equipment Contractors,2382.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1440 N. Dorgenois St, NEW ORLEANS, LA",29.976074,-90.075960,2.207100e+14
13665,398379,398379,9719897702,5/1/20,679,PPP,"NORTHLAKE MOVING &AMP; STORAGE, INC.",20252 HIGHWAY 36,COVINGTON,LA,...,999 Other,4842 Specialized Freight Trucking,4842.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"20252 HIGHWAY 36, COVINGTON, LA",30.482119,-90.065270,2.210304e+14
13666,398380,398380,9806418300,1/31/21,679,PPS,A-PRO INDUSTRIES LLC,204 Cresthill Dr,Youngsville,LA,...,333 Machinery,3339 Other General Purpose Machinery Manufactu...,3339.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"204 Cresthill Dr, Youngsville, LA",30.112051,-92.009890,2.205500e+14


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,2660988802,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354
1,3162008008,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490
2,4904497002,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370
3,5467867110,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495
4,5529817004,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [19]:
fips_check.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/LA_fips_scraped.csv")

In [44]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/LA_fips_scraped.csv") 
fips_check_scraped.head() 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,384714,384714,1115657202,4/15/20,679,PPP,"UP PROFESSIONAL SOLUTIONS, LLC",2400 Veterans Memorial Blvd. Suite 400,...,999 Other,5613 Employment Services,5613.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,2.205102e+14
1,1,1,384715,384715,1623197104,4/10/20,679,PPP,"DYNAMIC INDUSTRIES, INC.",400 Poydras Street Ste 1800 0.0,...,333 Machinery,"3331 Agriculture, Construction, and Mining Mac...",3331.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,2.207101e+14
2,2,2,384716,384716,4241517309,4/29/20,679,PPP,"BLESSEY MARINE SERVICE, INC.",1515 River Oaks Road East,...,999 Other,4832 Inland Water Transportation,4832.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,2.205102e+14
3,3,3,384717,384717,5102977301,4/30/20,679,PPP,"JOHN H. CARTER CO., INC.",17630 Perkins Rd,...,332 Fabricated Metal Products,3329 Other Fabricated Metal Product Manufacturing,3329.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,100 to 249,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,2.203300e+14
4,4,4,384718,384718,5378948703,4/2/21,679,PPP,KIPP NEW ORLEANS INC,1307 Oretha Castle Haley Blvd Ste 302,...,999 Other,6111 Elementary and Secondary Schools,6111.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,2.207101e+14


In [45]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [46]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [47]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [48]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,1115657202,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,2.205102e+14
1,1623197104,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,2.207101e+14
2,4241517309,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,2.205102e+14
3,5102977301,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,2.203300e+14
4,5378948703,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,2.207101e+14
...,...,...,...,...,...
13663,9651717708,"1000 Chinaberry Dr, Bossier City, LA",32.584312,-93.731130,2.201501e+14
13664,9659377206,"1440 N. Dorgenois St, NEW ORLEANS, LA",29.976074,-90.075960,2.207100e+14
13665,9719897702,"20252 HIGHWAY 36, COVINGTON, LA",30.482119,-90.065270,2.210304e+14
13666,9806418300,"204 Cresthill Dr, Youngsville, LA",30.112051,-92.009890,2.205500e+14


In [50]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.count()#5503

LoanNumber    57
full_add      57
Lat           57
Long          57
FIPS_z         0
dtype: int64

<h4>FIPS Script

In [26]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [27]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('22')] #LA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/101.0.4951.41]


  0%|          | 0/2760 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [28]:
geo_id_list =geo_id 

Length Check

In [29]:
print(len(geo_id_list))

2760


Matching FIPS to DataFrame

In [30]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:2684] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [31]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_21089/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
4,5378948703,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,[220710139001017]
9,5559717010,"8710 Jefferson Highway, BATON ROUGE, LA",30.418731,-91.093841,[220330038044002]
10,5284787303,"5900 HIGHWAY 90 E, BROUSSARD, LA",30.105006,-91.944056,[220550014272004]
14,1639267204,"160 Boro Lane,, Franklin, LA",29.816018,-91.545455,[221010411002139]
15,6544817105,"950 West Causeway Approach 0.0, Mandeville, LA",30.381717,-90.095283,[221030403063018]
...,...,...,...,...,...
13636,3660448504,"251 Highway 21 Ste 200, Madisonville, LA",30.423469,-90.158533,[221030404013018]
13641,4890887206,"650 Poydras St suite 2050, New Orleans, LA",29.949007,-90.070089,[220710134001046]
13644,5233138303,"1765, BATON ROUGE, LA",30.451468,-91.187147,[220330051003024]
13656,7563608402,"8814 Veterans Memorial Blvd, Metairie, LA",30.004714,-90.224015,[220510213001003]


Remove brackets

In [32]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_21089/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
4,5378948703,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,220710139001017
9,5559717010,"8710 Jefferson Highway, BATON ROUGE, LA",30.418731,-91.093841,220330038044002
10,5284787303,"5900 HIGHWAY 90 E, BROUSSARD, LA",30.105006,-91.944056,220550014272004
14,1639267204,"160 Boro Lane,, Franklin, LA",29.816018,-91.545455,221010411002139
15,6544817105,"950 West Causeway Approach 0.0, Mandeville, LA",30.381717,-90.095283,221030403063018
...,...,...,...,...,...
13636,3660448504,"251 Highway 21 Ste 200, Madisonville, LA",30.423469,-90.158533,221030404013018
13641,4890887206,"650 Poydras St suite 2050, New Orleans, LA",29.949007,-90.070089,220710134001046
13644,5233138303,"1765, BATON ROUGE, LA",30.451468,-91.187147,220330051003024
13656,7563608402,"8814 Veterans Memorial Blvd, Metairie, LA",30.004714,-90.224015,220510213001003


In [33]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
295,7205207001,"619 N. Main Street, JENNINGS, LA",36.183605,-96.569840,NaN
600,4082007104,"1001 11TH ST, MONROE, LA",42.601124,-89.646645,NaN
1003,1231277105,"401 West 7th Street, Shreveport, LA",34.046416,-118.254554,NaN
1123,6556928303,"502 N 2nd St, Monroe, LA",39.655320,-91.728440,NaN
1317,5071027000,"509 N 2nd St, MONROE, LA",39.655323,-91.728350,NaN
1437,2674807701,"40201 HIGHWAY 190 E, SLIDELL, LA",31.239600,-99.091648,NaN
1685,5622717101,"107 HIGHWAY 90 W, NEW IBERIA, LA",30.045167,-94.887227,NaN
1686,6516038302,"107 Highway 90 W, New Iberia, LA",30.045167,-94.887227,NaN
1796,6772877005,"1505 N 19TH ST, MONROE, LA",42.593950,-89.640240,NaN
1869,7870737007,"324 E LOCKWOOD ST, COVINGTON, LA",39.080982,-84.516600,NaN


Parse for Merge

In [34]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
4,5378948703,220710139001017
9,5559717010,220330038044002
10,5284787303,220550014272004
14,1639267204,221010411002139
15,6544817105,221030403063018
...,...,...
13636,3660448504,221030404013018
13641,4890887206,220710134001046
13644,5233138303,220330051003024
13656,7563608402,220510213001003


Read in base data

In [35]:
#if from geocode file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_la.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/LA_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1"], axis =1) 
fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,384714,384714,1115657202,4/15/20,679,PPP,"UP PROFESSIONAL SOLUTIONS, LLC",2400 Veterans Memorial Blvd. Suite 400,KENNER,...,999 Other,5613 Employment Services,5613.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,2.205102e+14
1,1,384715,384715,1623197104,4/10/20,679,PPP,"DYNAMIC INDUSTRIES, INC.",400 Poydras Street Ste 1800 0.0,New Orleans,...,333 Machinery,"3331 Agriculture, Construction, and Mining Mac...",3331.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,2.207101e+14
2,2,384716,384716,4241517309,4/29/20,679,PPP,"BLESSEY MARINE SERVICE, INC.",1515 River Oaks Road East,harahan,...,999 Other,4832 Inland Water Transportation,4832.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,2.205102e+14
3,3,384717,384717,5102977301,4/30/20,679,PPP,"JOHN H. CARTER CO., INC.",17630 Perkins Rd,BATON ROUGE,...,332 Fabricated Metal Products,3329 Other Fabricated Metal Product Manufacturing,3329.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,100 to 249,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,2.203300e+14
4,4,384718,384718,5378948703,4/2/21,679,PPP,KIPP NEW ORLEANS INC,1307 Oretha Castle Haley Blvd Ste 302,New Orleans,...,999 Other,6111 Elementary and Secondary Schools,6111.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,NaN


Merge with MainFrame

In [36]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,384714,384714,1115657202,4/15/20,679,PPP,"UP PROFESSIONAL SOLUTIONS, LLC",2400 Veterans Memorial Blvd. Suite 400,KENNER,...,5613 Employment Services,5613.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,2.205102e+14,NaN
1,1,384715,384715,1623197104,4/10/20,679,PPP,"DYNAMIC INDUSTRIES, INC.",400 Poydras Street Ste 1800 0.0,New Orleans,...,"3331 Agriculture, Construction, and Mining Mac...",3331.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,2.207101e+14,NaN
2,2,384716,384716,4241517309,4/29/20,679,PPP,"BLESSEY MARINE SERVICE, INC.",1515 River Oaks Road East,harahan,...,4832 Inland Water Transportation,4832.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,2.205102e+14,NaN
3,3,384717,384717,5102977301,4/30/20,679,PPP,"JOHN H. CARTER CO., INC.",17630 Perkins Rd,BATON ROUGE,...,3329 Other Fabricated Metal Product Manufacturing,3329.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,100 to 249,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,2.203300e+14,NaN
4,4,384718,384718,5378948703,4/2/21,679,PPP,KIPP NEW ORLEANS INC,1307 Oretha Castle Haley Blvd Ste 302,New Orleans,...,6111 Elementary and Secondary Schools,6111.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,NaN,220710139001017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13663,13663,398377,398377,9651717708,5/1/20,679,PPP,"QUALITY EXTERIORS, INC.",1000 Chinaberry Dr,Bossier City,...,"2381 Foundation, Structure, and Building Exter...",2381.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1000 Chinaberry Dr, Bossier City, LA",32.584312,-93.731130,2.201501e+14,NaN
13664,13664,398378,398378,9659377206,4/28/20,679,PPP,E-1 ELECTRIC LLC,1440 N. Dorgenois St,NEW ORLEANS,...,2382 Building Equipment Contractors,2382.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1440 N. Dorgenois St, NEW ORLEANS, LA",29.976074,-90.075960,2.207100e+14,NaN
13665,13665,398379,398379,9719897702,5/1/20,679,PPP,"NORTHLAKE MOVING &AMP; STORAGE, INC.",20252 HIGHWAY 36,COVINGTON,...,4842 Specialized Freight Trucking,4842.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"20252 HIGHWAY 36, COVINGTON, LA",30.482119,-90.065270,2.210304e+14,NaN
13666,13666,398380,398380,9806418300,1/31/21,679,PPS,A-PRO INDUSTRIES LLC,204 Cresthill Dr,Youngsville,...,3339 Other General Purpose Machinery Manufactu...,3339.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"204 Cresthill Dr, Youngsville, LA",30.112051,-92.009890,2.205500e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [37]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,384714,384714,1115657202,4/15/20,679,PPP,"UP PROFESSIONAL SOLUTIONS, LLC",2400 Veterans Memorial Blvd. Suite 400,KENNER,...,5613.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,2.205102e+14,NaN,220510205062018.0
1,1,384715,384715,1623197104,4/10/20,679,PPP,"DYNAMIC INDUSTRIES, INC.",400 Poydras Street Ste 1800 0.0,New Orleans,...,3331.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,2.207101e+14,NaN,220710134004025.0
2,2,384716,384716,4241517309,4/29/20,679,PPP,"BLESSEY MARINE SERVICE, INC.",1515 River Oaks Road East,harahan,...,4832.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,2.205102e+14,NaN,220510242023039.0
3,3,384717,384717,5102977301,4/30/20,679,PPP,"JOHN H. CARTER CO., INC.",17630 Perkins Rd,BATON ROUGE,...,3329.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,100 to 249,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,2.203300e+14,NaN,220330040212030.0
4,4,384718,384718,5378948703,4/2/21,679,PPP,KIPP NEW ORLEANS INC,1307 Oretha Castle Haley Blvd Ste 302,New Orleans,...,6111.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,NaN,220710139001017,220710139001017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13663,13663,398377,398377,9651717708,5/1/20,679,PPP,"QUALITY EXTERIORS, INC.",1000 Chinaberry Dr,Bossier City,...,2381.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1000 Chinaberry Dr, Bossier City, LA",32.584312,-93.731130,2.201501e+14,NaN,220150111193014.0
13664,13664,398378,398378,9659377206,4/28/20,679,PPP,E-1 ELECTRIC LLC,1440 N. Dorgenois St,NEW ORLEANS,...,2382.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1440 N. Dorgenois St, NEW ORLEANS, LA",29.976074,-90.075960,2.207100e+14,NaN,220710036001007.0
13665,13665,398379,398379,9719897702,5/1/20,679,PPP,"NORTHLAKE MOVING &AMP; STORAGE, INC.",20252 HIGHWAY 36,COVINGTON,...,4842.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"20252 HIGHWAY 36, COVINGTON, LA",30.482119,-90.065270,2.210304e+14,NaN,221030406081008.0
13666,13666,398380,398380,9806418300,1/31/21,679,PPS,A-PRO INDUSTRIES LLC,204 Cresthill Dr,Youngsville,...,3339.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"204 Cresthill Dr, Youngsville, LA",30.112051,-92.009890,2.205500e+14,NaN,220550014178011.0


Drop Extra FIPS Columns

In [38]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,384714,384714,1115657202,4/15/20,679,PPP,"UP PROFESSIONAL SOLUTIONS, LLC",2400 Veterans Memorial Blvd. Suite 400,KENNER,...,999 Other,5613 Employment Services,5613.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,220510205062018.0
1,1,384715,384715,1623197104,4/10/20,679,PPP,"DYNAMIC INDUSTRIES, INC.",400 Poydras Street Ste 1800 0.0,New Orleans,...,333 Machinery,"3331 Agriculture, Construction, and Mining Mac...",3331.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,220710134004025.0
2,2,384716,384716,4241517309,4/29/20,679,PPP,"BLESSEY MARINE SERVICE, INC.",1515 River Oaks Road East,harahan,...,999 Other,4832 Inland Water Transportation,4832.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,220510242023039.0
3,3,384717,384717,5102977301,4/30/20,679,PPP,"JOHN H. CARTER CO., INC.",17630 Perkins Rd,BATON ROUGE,...,332 Fabricated Metal Products,3329 Other Fabricated Metal Product Manufacturing,3329.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,100 to 249,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,220330040212030.0
4,4,384718,384718,5378948703,4/2/21,679,PPP,KIPP NEW ORLEANS INC,1307 Oretha Castle Haley Blvd Ste 302,New Orleans,...,999 Other,6111 Elementary and Secondary Schools,6111.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,220710139001017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13663,13663,398377,398377,9651717708,5/1/20,679,PPP,"QUALITY EXTERIORS, INC.",1000 Chinaberry Dr,Bossier City,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1000 Chinaberry Dr, Bossier City, LA",32.584312,-93.731130,220150111193014.0
13664,13664,398378,398378,9659377206,4/28/20,679,PPP,E-1 ELECTRIC LLC,1440 N. Dorgenois St,NEW ORLEANS,...,999 Other,2382 Building Equipment Contractors,2382.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1440 N. Dorgenois St, NEW ORLEANS, LA",29.976074,-90.075960,220710036001007.0
13665,13665,398379,398379,9719897702,5/1/20,679,PPP,"NORTHLAKE MOVING &AMP; STORAGE, INC.",20252 HIGHWAY 36,COVINGTON,...,999 Other,4842 Specialized Freight Trucking,4842.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"20252 HIGHWAY 36, COVINGTON, LA",30.482119,-90.065270,221030406081008.0
13666,13666,398380,398380,9806418300,1/31/21,679,PPS,A-PRO INDUSTRIES LLC,204 Cresthill Dr,Youngsville,...,333 Machinery,3339 Other General Purpose Machinery Manufactu...,3339.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"204 Cresthill Dr, Youngsville, LA",30.112051,-92.009890,220550014178011.0


In [39]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
295,295,385009,385009,7205207001,4/7/20,679,PPP,"THERAPY CENTER OF JEFFERSON DAVIS PARISH, INC.",619 N. Main Street,JENNINGS,...,999 Other,6213 Offices of Other Health Practitioners,6213.0,LOUISIANA DISTRICT OFFICE,d $2-5 million,100 to 249,"619 N. Main Street, JENNINGS, LA",36.183605,-96.569840,NaN
600,600,385314,385314,4082007104,4/12/20,679,PPP,"SOUTHEAST FOODS, LLC",1001 11TH ST,MONROE,...,999 Other,4451 Grocery Stores,4451.0,LOUISIANA DISTRICT OFFICE,c $1-2 million,250 to 499,"1001 11TH ST, MONROE, LA",42.601124,-89.646645,NaN
1003,1003,385717,385717,1231277105,4/10/20,679,PPP,"SHREVEPORT CHARTER SCHOOL, INC",401 West 7th Street,Shreveport,...,999 Other,6111 Elementary and Secondary Schools,6111.0,LOUISIANA DISTRICT OFFICE,c $1-2 million,100 to 249,"401 West 7th Street, Shreveport, LA",34.046416,-118.254554,NaN
1123,1123,385837,385837,6556928303,1/27/21,679,PPS,ASCENT HEALTH INC.,502 N 2nd St,Monroe,...,999 Other,6241 Individual and Family Services,6241.0,LOUISIANA DISTRICT OFFICE,c $1-2 million,100 to 249,"502 N 2nd St, Monroe, LA",39.655320,-91.728440,NaN
1317,1317,386031,386031,5071027000,4/5/20,679,PPP,CARE SOLUTIONS INC,509 N 2nd St,MONROE,...,999 Other,6216 Home Health Care Services,6216.0,LOUISIANA DISTRICT OFFICE,c $1-2 million,250 to 499,"509 N 2nd St, MONROE, LA",39.655323,-91.728350,NaN
1437,1437,386151,386151,2674807701,5/1/20,679,PPP,"STARC OF LOUISIANA, INC",40201 HIGHWAY 190 E,SLIDELL,...,999 Other,6232 Residential Intellectual and Developmenta...,6232.0,LOUISIANA DISTRICT OFFICE,c $1-2 million,250 to 499,"40201 HIGHWAY 190 E, SLIDELL, LA",31.239600,-99.091648,NaN
1685,1685,386399,386399,5622717101,4/13/20,679,PPP,SEATRAN MARINE LLC,107 HIGHWAY 90 W,NEW IBERIA,...,999 Other,5324 Commercial and Industrial Machinery and E...,5324.0,LOUISIANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"107 HIGHWAY 90 W, NEW IBERIA, LA",30.045167,-94.887227,NaN
1686,1686,386400,386400,6516038302,1/27/21,679,PPS,SEATRAN MARINE LLC,107 Highway 90 W,New Iberia,...,999 Other,4883 Support Activities for Water Transportation,4883.0,LOUISIANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"107 Highway 90 W, New Iberia, LA",30.045167,-94.887227,NaN
1796,1796,386510,386510,6772877005,4/7/20,679,PPP,"DEAN MORRIS, LLC",1505 N 19TH ST,MONROE,...,999 Other,5411 Legal Services,5411.0,LOUISIANA DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"1505 N 19TH ST, MONROE, LA",42.593950,-89.640240,NaN
1869,1869,386583,386583,7870737007,4/8/20,679,PPP,BAYOU'S BEST BURGERS LLC,324 E LOCKWOOD ST,COVINGTON,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,LOUISIANA DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"324 E LOCKWOOD ST, COVINGTON, LA",39.080982,-84.516600,NaN


Convert to File

In [40]:
fips_MO = fips_merge_drop
#fips_TN = fips_merge_drop

In [41]:
fips_MO.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/LA_fips_scraped.csv") 

Review & Compare

In [42]:
fips_MO = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/LA_fips_scraped.csv") 
fips_MO

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,384714,384714,1115657202,4/15/20,679,PPP,"UP PROFESSIONAL SOLUTIONS, LLC",2400 Veterans Memorial Blvd. Suite 400,...,999 Other,5613 Employment Services,5613.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"2400 Veterans Memorial Blvd. Suite 400, KENNER...",30.006386,-90.235860,2.205102e+14
1,1,1,384715,384715,1623197104,4/10/20,679,PPP,"DYNAMIC INDUSTRIES, INC.",400 Poydras Street Ste 1800 0.0,...,333 Machinery,"3331 Agriculture, Construction, and Mining Mac...",3331.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"400 Poydras Street Ste 1800 0.0, New Orleans, LA",29.948801,-90.067540,2.207101e+14
2,2,2,384716,384716,4241517309,4/29/20,679,PPP,"BLESSEY MARINE SERVICE, INC.",1515 River Oaks Road East,...,999 Other,4832 Inland Water Transportation,4832.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1515 River Oaks Road East, harahan, LA",29.961603,-90.200820,2.205102e+14
3,3,3,384717,384717,5102977301,4/30/20,679,PPP,"JOHN H. CARTER CO., INC.",17630 Perkins Rd,...,332 Fabricated Metal Products,3329 Other Fabricated Metal Product Manufacturing,3329.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,100 to 249,"17630 Perkins Rd, BATON ROUGE, LA",30.346773,-91.031660,2.203300e+14
4,4,4,384718,384718,5378948703,4/2/21,679,PPP,KIPP NEW ORLEANS INC,1307 Oretha Castle Haley Blvd Ste 302,...,999 Other,6111 Elementary and Secondary Schools,6111.0,LOUISIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1307 Oretha Castle Haley Blvd Ste 302, New Orl...",29.942532,-90.078461,2.207101e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13663,13663,13663,398377,398377,9651717708,5/1/20,679,PPP,"QUALITY EXTERIORS, INC.",1000 Chinaberry Dr,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1000 Chinaberry Dr, Bossier City, LA",32.584312,-93.731130,2.201501e+14
13664,13664,13664,398378,398378,9659377206,4/28/20,679,PPP,E-1 ELECTRIC LLC,1440 N. Dorgenois St,...,999 Other,2382 Building Equipment Contractors,2382.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1440 N. Dorgenois St, NEW ORLEANS, LA",29.976074,-90.075960,2.207100e+14
13665,13665,13665,398379,398379,9719897702,5/1/20,679,PPP,"NORTHLAKE MOVING &AMP; STORAGE, INC.",20252 HIGHWAY 36,...,999 Other,4842 Specialized Freight Trucking,4842.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"20252 HIGHWAY 36, COVINGTON, LA",30.482119,-90.065270,2.210304e+14
13666,13666,13666,398380,398380,9806418300,1/31/21,679,PPS,A-PRO INDUSTRIES LLC,204 Cresthill Dr,...,333 Machinery,3339 Other General Purpose Machinery Manufactu...,3339.0,LOUISIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"204 Cresthill Dr, Youngsville, LA",30.112051,-92.009890,2.205500e+14
